In [55]:
import pandas as pd

In [56]:
# Dropping unnecessary columns 
def drop_unnecessary_columns(df):
    return df.drop(['case_enquiry_id', 'case_status', 'closure_reason', 'type', 'queue',
             'submittedphoto', 'closedphoto', 'precinct', 'neighborhood_services_district',
             'ward', 'case_title', 'target_dt', 'closed_dt', 'location'], axis=1)

# Dropping unnecessary rows
def drop_unnecessary_rows(df):
    return df.dropna(subset=['city_council_district', 'fire_district', 'pwd_district', 'neighborhood'])

# Formatting dates
def format_dates(df):
    df['open_dt'] = pd.to_datetime(df['open_dt'])

    df['open_year'] = df['open_dt'].dt.year
    df['open_month'] = df['open_dt'].dt.month
    df['open_day'] = df['open_dt'].dt.day
    df['open_weekday'] = df['open_dt'].dt.day_name()

    df = df.drop('open_dt', axis=1)
    
    return df

# Reorder columns 
def reorder_columns(df):
    return df.reindex(columns=['open_year', 'open_month', 'open_day', 'open_weekday', 
                               'ontime', 'subject', 'reason', 'department', 'fire_district',
                                                   'pwd_district', 'city_council_district', 'police_district',
                                                   'neighborhood', 'location_street_name', 'location_zipcode', 
                                                   'latitude', 'longitude', 'source'])

# Clean df
def clean_df(df):
    
    df = drop_unnecessary_columns(df)
    df = drop_unnecessary_rows(df)
    df = format_dates(df)
    df = reorder_columns(df)
    
    return df

In [57]:
files_311 = ['311_service_requests_2015.csv', '311_service_requests_2016.csv', '311_service_requests_2017.csv',
             '311_service_requests_2018.csv', '311_service_requests_2019.csv']

dfs_311 = [pd.read_csv(file) for file in files_311]

cleaned_dfs_311 = [clean_df(df) for df in dfs_311]

combined_dfs_311 = pd.concat(cleaned_dfs_311, ignore_index=True)

sorted_df_311 = combined_dfs_311.sort_values(by=['open_year', 'open_month', 'open_day'], 
                                             ascending=[True, True, True])

In [58]:
sorted_df_311.head()

,open_year,open_month,open_day,open_weekday,ontime,subject,reason,department,fire_district,pwd_district,city_council_district,police_district,neighborhood,location_street_name,location_zipcode,latitude,longitude,source
0,2015,1,1,Thursday,ONTIME,Mayor's 24 Hour Hotline,Notification,INFO,3,1B,8,A1,Downtown / Financial District,33 Beacon St,2108.0,42.3574,-71.0648,Constituent Call
1,2015,1,1,Thursday,ONTIME,Public Works Department,Street Lights,PWDx,4,10A,8,D4,Fenway / Kenmore / Audubon Circle / Longwood,145-151 Ipswich St,2215.0,42.3474,-71.0944,Citizens Connect App
2,2015,1,1,Thursday,OVERDUE,Public Works Department,Street Lights,PWDx,4,10A,8,D4,Fenway / Kenmore / Audubon Circle / Longwood,145-151 Ipswich St,2215.0,42.3474,-71.0944,Citizens Connect App
3,2015,1,1,Thursday,ONTIME,Animal Control,Animal Issues,ANML,12,06,6,E5,West Roxbury,4833 Washington St,2132.0,42.2711,-71.1464,Constituent Call
4,2015,1,1,Thursday,ONTIME,Mayor's 24 Hour Hotline,Notification,INFO,6,03,2,C6,Dorchester,10 Washburn St,2125.0,42.3254,-71.0574,Constituent Call


In [64]:
def rename(col):
    if col == 'Downtown / Financial District':
        return 'Downtown'
    if col == ' ':
        return 'Boston'
    if col == 'Greater Mattapan':
        return 'Mattapan'
    return col

In [65]:
sorted_df_311['neighborhood'] = sorted_df_311['neighborhood'].apply(lambda x: rename(x))

In [66]:
sorted_df_311['neighborhood'].unique()

array(['Downtown', 'Fenway / Kenmore / Audubon Circle / Longwood',
       'West Roxbury', 'Dorchester', 'Hyde Park', 'Roxbury', 'South End',
       'Mission Hill', 'Back Bay', 'Charlestown', 'Beacon Hill',
       'Allston / Brighton', 'Boston', 'East Boston', 'South Boston',
       'Jamaica Plain', 'Brighton',
       'South Boston / South Boston Waterfront', 'Roslindale', 'Mattapan',
       'Allston', 'Chestnut Hill'], dtype=object)

In [67]:
sorted_df_311.to_csv("merged_311.csv")